In [1]:
import pandas as pd

df = pd.read_csv('germany_state_data.csv')

df.head()

,state_id,state_name,state_flag_image_url,geom
0,Baden-Württemberg,Baden-Württemberg,https://raw.githubusercontent.com/Prasadmadhus...,NaN
1,Bavaria,Bavaria,https://raw.githubusercontent.com/Prasadmadhus...,NaN
2,Berlin,Berlin,https://raw.githubusercontent.com/Prasadmadhus...,NaN
3,Brandenburg,Brandenburg,https://raw.githubusercontent.com/Prasadmadhus...,NaN
4,Bremen,Bremen,https://raw.githubusercontent.com/Prasadmadhus...,NaN


In [2]:
df['state_name'].tolist()

['Baden-Württemberg',
 'Bavaria',
 'Berlin',
 'Brandenburg',
 'Bremen',
 'Hamburg',
 'Hesse',
 'Lower Saxony',
 'Mecklenburg-Vorpommern',
 'North Rhine-Westphalia',
 'Rhineland-Palatinate',
 'Saarland',
 'Saxony',
 'Saxony-Anhalt',
 'Schleswig-Holstein',
 'Thuringia']

In [3]:
import geopandas as gpd

gdf = gpd.read_file('germany_states_geometry.json')

gdf.head()

,id,name,nameEN,short,geometry
0,08,Baden-Württemberg,None,BW,"MULTIPOLYGON (((8.70269 47.71478, 8.67436 47.7..."
1,09,Bayern,Bavaria,BY,"POLYGON ((10.45441 47.55591, 10.47292 47.55128..."
2,11,Berlin,None,BE,"POLYGON ((13.16621 52.39413, 13.15979 52.4027,..."
3,12,Brandenburg,None,BR,"POLYGON ((14.41221 53.3293, 14.39937 53.33, 14..."
4,04,Bremen,None,HB,"MULTIPOLYGON (((8.98445 53.12619, 8.95688 53.1..."


In [4]:
gdf['name'].tolist()

['Baden-Württemberg',
 'Bayern',
 'Berlin',
 'Brandenburg',
 'Bremen',
 'Hamburg',
 'Hessen',
 'Mecklenburg-Vorpommern',
 'Niedersachsen',
 'Nordrhein-Westfalen',
 'Rheinland-Pfalz',
 'Saarland',
 'Sachsen-Anhalt',
 'Sachsen',
 'Schleswig-Holstein',
 'Thüringen']

In [5]:
name_mapping = {
    'Bayern': 'Bavaria',
    'Hessen': 'Hesse',
    'Niedersachsen': 'Lower Saxony',
    'Nordrhein-Westfalen': 'North Rhine-Westphalia',
    'Rheinland-Pfalz': 'Rhineland-Palatinate',
    'Sachsen': 'Saxony',
    'Sachsen-Anhalt': 'Saxony-Anhalt',
    'Thüringen': 'Thuringia'
    # The rest of the names are the same, no need to include them
}

gdf['name'] = gdf['name'].replace(name_mapping)

gdf['name'].tolist()

['Baden-Württemberg',
 'Bavaria',
 'Berlin',
 'Brandenburg',
 'Bremen',
 'Hamburg',
 'Hesse',
 'Mecklenburg-Vorpommern',
 'Lower Saxony',
 'North Rhine-Westphalia',
 'Rhineland-Palatinate',
 'Saarland',
 'Saxony-Anhalt',
 'Saxony',
 'Schleswig-Holstein',
 'Thuringia']

In [6]:
merged_gdf = gdf.merge(
    df[["state_id", "state_name", "state_flag_image_url"]],
    left_on="name",          # column in gdf
    right_on="state_name",   # column in df
    how="left"
)
merged_gdf = gpd.GeoDataFrame(
    merged_gdf,
    geometry="geometry",
    crs=gdf.crs
)
merged_gdf.head()

,id,name,nameEN,short,geometry,state_id,state_name,state_flag_image_url
0,08,Baden-Württemberg,None,BW,"MULTIPOLYGON (((8.70269 47.71478, 8.67436 47.7...",Baden-Württemberg,Baden-Württemberg,https://raw.githubusercontent.com/Prasadmadhus...
1,09,Bavaria,Bavaria,BY,"POLYGON ((10.45441 47.55591, 10.47292 47.55128...",Bavaria,Bavaria,https://raw.githubusercontent.com/Prasadmadhus...
2,11,Berlin,None,BE,"POLYGON ((13.16621 52.39413, 13.15979 52.4027,...",Berlin,Berlin,https://raw.githubusercontent.com/Prasadmadhus...
3,12,Brandenburg,None,BR,"POLYGON ((14.41221 53.3293, 14.39937 53.33, 14...",Brandenburg,Brandenburg,https://raw.githubusercontent.com/Prasadmadhus...
4,04,Bremen,None,HB,"MULTIPOLYGON (((8.98445 53.12619, 8.95688 53.1...",Bremen,Bremen,https://raw.githubusercontent.com/Prasadmadhus...


In [7]:
merged_gdf = merged_gdf.drop(columns=["id", "name", "nameEN", "short"])
merged_gdf.head()

,geometry,state_id,state_name,state_flag_image_url
0,"MULTIPOLYGON (((8.70269 47.71478, 8.67436 47.7...",Baden-Württemberg,Baden-Württemberg,https://raw.githubusercontent.com/Prasadmadhus...
1,"POLYGON ((10.45441 47.55591, 10.47292 47.55128...",Bavaria,Bavaria,https://raw.githubusercontent.com/Prasadmadhus...
2,"POLYGON ((13.16621 52.39413, 13.15979 52.4027,...",Berlin,Berlin,https://raw.githubusercontent.com/Prasadmadhus...
3,"POLYGON ((14.41221 53.3293, 14.39937 53.33, 14...",Brandenburg,Brandenburg,https://raw.githubusercontent.com/Prasadmadhus...
4,"MULTIPOLYGON (((8.98445 53.12619, 8.95688 53.1...",Bremen,Bremen,https://raw.githubusercontent.com/Prasadmadhus...


In [8]:
merged_gdf.to_file(
    "germany_states_final.geojson",
    driver="GeoJSON",
    encoding="utf-8"
)